In [1]:
# !pip install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import logging
import multiprocessing
from datetime import datetime, timedelta

from src import modules as f
%load_ext autoreload
%autoreload 2

In [13]:
# Download, load, extract, model...
interval = '1d'
stock_set = f.download_interval_process(interval, 1)
print(f'Stock set: \n{stock_set}\n')

Stock set: 
{'CSCO', 'EXE', 'DIS', 'DECK', 'WMB', 'ABBV', 'RCL', 'BAC', 'MRK', 'VST', 'T', 'EOG', 'WMT', 'PM', 'PLTR', 'CRM', 'EXC', 'XEL', 'LLY', 'AMZN', 'LNG', 'GOOGL', 'NEE', 'FCNCA', 'SW', 'PGR', 'JPM', 'MSFT', 'TSLA', 'D', 'WFC', 'DUK', 'COST', 'HD', 'TDG', 'WSM', 'PEP', 'SPGI', 'CVX', 'MDLZ', 'TGT', 'MA', 'TMUS', 'AAPL', 'ORCL', 'XOM', 'KO', 'ISRG', 'ACN', 'SO', '^VIX', 'COP', 'ADBE', 'WELL', 'FWONK', 'APH', 'OKE', 'GS', 'PG', 'SLB', 'NRG', 'AJG', 'TPR', 'V', 'META', 'PSX', 'UNH', 'CMCSA', 'DASH', 'IBM', 'GOOG', 'CEG', 'AXP', 'NVDA', 'DKNG', 'AEP', 'AMGN', 'MO', 'KMI', 'TMO', 'ABT', 'VZ', 'MSI', 'AVGO', 'CL', 'NFLX', 'JNJ', 'BSX', 'BRK-B', 'TTWO', 'SRE', 'ATO'}



In [14]:
vix_df = f.load_raw('^VIX', '1d')
vix_df.tail(5)

,open,high,low,close,adj_close,volume
Date,,,,,,
2025-03-31 00:00:00-05:00,24.110001,24.799999,21.67,22.280001,22.280001,0
2025-04-01 00:00:00-05:00,22.059999,23.520000,21.58,21.770000,21.770000,0
2025-04-02 00:00:00-05:00,22.299999,23.660000,20.68,21.510000,21.510000,0
2025-04-03 00:00:00-05:00,26.379999,30.020000,24.93,30.020000,30.020000,0
2025-04-04 00:00:00-05:00,30.120001,30.770000,29.99,30.309999,30.309999,0


In [16]:
f.make_table_features_process(stock_set, interval, 8)

KeyboardInterrupt: 

In [ ]:
f.make_master_table(stock_set, interval)

In [ ]:
f.xg_boost_model(interval='1d', grid_search_on=True)

In [15]:
symbol = 'TSLA'.upper()
interval = '1d'
f.download(symbol='^VIX', interval=interval)
f.model_prospect(symbol, interval)
df_val = f.model_validation(symbol, interval)
correctly_predicted = df_val.dir_pred_match.sum()
print(f'\nCorrectly Predicted: {correctly_predicted}')
entries_predicted = df_val.shape[0]
print(f'Entries Predicted: {entries_predicted}')
print(f'Percent Correct: {correctly_predicted / entries_predicted}')


Model Prediction 1d TSLA:
Predicted Next 1d Movement: DOWN

Model Prediction Probabilities:
no_change (0): 0.3038
up (1): 0.3410
down (2): 0.3552

Last Entry 1d TSLA Datetime Used for Prediction:
NOTE: It's in or contains the full 1d time interval.
EST: 2025-01-22 12:00:00 AM EST-0500
PDT: 2025-01-21 09:00:00 PM PST-0800

Correctly Predicted: 12
Entries Predicted: 50
Percent Correct: 0.24
